In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
print('Dependencies Imported')

Dependencies Imported


In [3]:
data = pd.read_csv('Dataset/Coursera.csv')


In [4]:
data = data[['Course Name','Difficulty Level','Course Description','Skills']]


# Data Pre-Processing

In [5]:
data['Course Name'] = data['Course Name'].str.replace(' ',',')
data['Course Name'] = data['Course Name'].str.replace(',,',',')
data['Course Name'] = data['Course Name'].str.replace(':','')
data['Course Description'] = data['Course Description'].str.replace(' ',',')
data['Course Description'] = data['Course Description'].str.replace(',,',',')
data['Course Description'] = data['Course Description'].str.replace('_','')
data['Course Description'] = data['Course Description'].str.replace(':','')
data['Course Description'] = data['Course Description'].str.replace('(','')
data['Course Description'] = data['Course Description'].str.replace(')','')

#removing paranthesis from skills columns 
data['Skills'] = data['Skills'].str.replace('(','')
data['Skills'] = data['Skills'].str.replace(')','')

In [6]:
data.head(5)


,Course Name,Difficulty Level,Course Description,Skills
0,"Write,A,Feature,Length,Screenplay,For,Film,Or,...",Beginner,"Write,a,Full,Length,Feature,Film,Script,In,thi...",Drama Comedy peering screenwriting film D...
1,"Business,Strategy,Business,Model,Canvas,Analys...",Beginner,"By,the,end,of,this,guided,project,you,will,be,...",Finance business plan persona user experienc...
2,"Silicon,Thin,Film,Solar,Cells",Advanced,"This,course,consists,of,a,general,presentation...",chemistry physics Solar Energy film lambda...
3,"Finance,for,Managers",Intermediate,"When,it,comes,to,numbers,there,is,always,more,...",accounts receivable dupont analysis analysis...
4,"Retrieve,Data,using,Single-Table,SQL,Queries",Beginner,"In,this,course,you�ll,learn,how,to,effectively...",Data Analysis select sql database management...


# Tags Column

The tags column is the combination of the following columns : Course Name + Difficulty Level + Course Description + Skills

In [23]:
data['tags'] = data['Course Name'] + data['Difficulty Level'] + data['Course Description'] + data['Skills']
data['tags'][0]

'Write,A,Feature,Length,Screenplay,For,Film,Or,TelevisionBeginnerWrite,a,Full,Length,Feature,Film,Script,In,this,course,you,will,write,a,complete,feature-length,screenplay,for,film,or,television,be,it,a,serious,drama,or,romantic,comedy,or,anything,in,between.,You�ll,learn,to,break,down,the,creative,process,into,components,and,you�ll,discover,a,structured,process,that,allows,you,to,produce,a,polished,and,pitch-ready,script,by,the,end,of,the,course.,Completing,this,project,will,increase,your,confidence,in,your,ideas,and,abilities,and,you�ll,feel,prepared,to,pitch,your,first,script,and,get,started,on,your,next.,This,is,a,course,designed,to,tap,into,your,creativity,and,is,based,in,"Active,Learning".,Most,of,the,actual,learning,takes,place,within,your,own,activities,-,that,is,writing!,You,will,learn,by,doing.,Here,is,a,link,to,a,TRAILER,for,the,course.,To,view,the,trailer,please,copy,and,paste,the,link,into,your,browser.,https//vimeo.com/382067900/b78b800dc0,Learner,review,"Love,the,approac

# Dataframe 

In [8]:
new_df = data[['Course Name','tags']]


In [9]:
new_df['tags'] = data['tags'].str.replace(',',' ')


In [10]:
new_df['Course Name'] = data['Course Name'].str.replace(',',' ')

In [11]:
new_df.rename(columns = {'Course Name':'course_name'}, inplace = True)


In [12]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower()) #lower casing the tags column


In [13]:
new_df.head(5)


,course_name,tags
0,Write A Feature Length Screenplay For Film Or ...,write a feature length screenplay for film or ...
1,Business Strategy Business Model Canvas Analys...,business strategy business model canvas analys...
2,Silicon Thin Film Solar Cells,silicon thin film solar cellsadvancedthis cour...
3,Finance for Managers,finance for managersintermediatewhen it comes ...
4,Retrieve Data using Single-Table SQL Queries,retrieve data using single-table sql queriesbe...


# Text Vectorization

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()


In [15]:
import nltk #for stemming process
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [16]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [17]:
new_df['tags'] = new_df['tags'].apply(stem) 


In [18]:
from sklearn.metrics.pairwise import cosine_similarity


In [19]:
similarity = cosine_similarity(vectors)


In [26]:
def recommend(course):
    course_index = new_df[new_df['course_name'] == course].index[0]
    distances = similarity[course_index]
    course_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:6]
    
    for i in course_list:
        print(new_df.iloc[i[0]].course_name)

In [30]:
crs = 'The Music of American English Pronunciation'
print("Course similar to",crs,": " )
recommend(crs) 


Course similar to The Music of American English Pronunciation : 
Consonants of American English Pronunciation
Vowels of American English Pronunciation
Tricky American English Pronunciation
The Pronunciation of American English Project
Teaching Tips for Tricky English Grammar
